In [1]:
import os

In [2]:
%pwd

'e:\\Data_Science_Project\\StockScout\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    ticker: str
    local_data_file: Path


In [9]:
from src.StockScout.constants import *
from src.StockScout.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            ticker=config.ticker,
            local_data_file=config.local_data_file
        )

        return data_ingestion_config

In [16]:
from src.StockScout import logger
from src.StockScout.config.configuration import DataIngestionConfig
import yfinance as yf
import os

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):

        # Download AAPL data 
        logger.info(f"{self.config.ticker}'s data download started")
        data = yf.download(self.config.ticker)
        logger.info(f"{self.config.ticker}'s data downloaded")


        data.columns = data.columns.droplevel(1)
        data = data['Close']
        # Set filename 
        filename = 'data.csv'
        filepath = os.path.join(self.config.local_data_file, filename)

        # Check if file exists, if so delete it
        if os.path.isfile(filepath):
            os.remove(filepath)
            
        # Save downloaded data to csv    
        data.to_csv(filepath)

        logger.info(f"File downloaded and saved to: {filepath}")


    

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2025-01-28 14:19:40,329: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-28 14:19:40,333: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-28 14:19:40,337: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-28 14:19:40,339: INFO: common: created directory at: artifacts]
[2025-01-28 14:19:40,342: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-28 14:19:40,344: INFO: 2252133005: AAPL's data download started]


[*********************100%***********************]  1 of 1 completed

[2025-01-28 14:19:41,972: INFO: 2252133005: AAPL's data downloaded]
[2025-01-28 14:19:42,017: INFO: 2252133005: File downloaded and saved to: artifacts/data_ingestion\data.csv]
